In [1]:
import pandas as pd
from sqlalchemy import create_engine, URL

<h3>Loading Dataset</h3>

In [2]:
df = pd.read_csv("Google-Playstore.csv")

In [ ]:
#Columns With Missing Values
df.count()[df.count()<df.shape[0]]

In [3]:
df1 = df.copy()
#Drop Missing Values
#df.dropna(inplace=True)

#Drop Duplicates
df1.drop_duplicates(inplace=True)


#Columns to consider for Drop missing values
df1.dropna(subset=['Developer Id', 'App Name', 'Installs', 'Minimum Installs'], inplace=True)

#Filling Missing Values on some Columns is preffered over dropping the entire row
cols_to_be_filled = ['Privacy Policy','Developer Website', 'Developer Email','Currency',
                     'Rating', 'Rating Count', 'Minimum Android']


df1['Size'] = pd.to_numeric(df['Size'].str.removesuffix("M"), errors='coerce', downcast='float').fillna(0)
#Fill Missing Values in Released Column with Last Update
#Missing Dates in Released Column
missing_released = df['Released'].isna()
df1.loc[missing_released,'Released'] = df.loc[missing_released,'Last Updated']

#Converting date fields to datetime
df1['Released'] = pd.to_datetime(df['Released'], format= "%b %d, %Y")
df1['Last Updated'] = pd.to_datetime(df['Last Updated'], format= "%b %d, %Y")

import numpy as np
#Generate Category Id for Categories Table
category_id = { c:i for i,c in enumerate(df['Category'].unique(),1)}
df1['Category Id'] = df['Category'].map(category_id)

#Exporting the preprocessed Dataset
df1.to_csv("preprocessed_data.csv",index=False)

<h3>Connect To Database</h3>

In [ ]:
#df1 = pd.read_csv('preprocessed_data.csv')

In [4]:
database_columns = {col:col.lower().replace(" ","_") for col in df1.columns}
df1.rename(columns=database_columns, inplace=True)

In [5]:
# Create Engine for PostgreSQL
url_object = URL.create(
    "postgresql+psycopg2",
    username="postgres",
    password="123456",  # plain (unescaped) text
    host="localhost",
    database="googleplay",
    port=5432
    )

engine = create_engine(url_object)
db = engine.connect()

In [6]:
developers_columns = [
    "developer_id",         # Primary Key
    "developer_website",    # Developer's website
    "developer_email"       # Developer's email
]

categories_columns = [
    "category_id",          # Primary Key
    "category"              # Category name
]

apps_columns = [
    "app_id",               # Primary Key
    "app_name",             # App name
    "developer_id",         # Foreign Key referencing developers(developer_id)
    "category_id",          # Foreign Key referencing categories(category_id)
    "rating",               # App rating
    "rating_count",         # Number of ratings received
    "installs",             # Install count as a string (e.g., "1M+")
    "minimum_installs",     # Minimum number of installs
    "maximum_installs",     # Maximum number of installs
    "minimum_android",
    "free",                 # Whether the app is free or not
    "price",                # Price of the app (if applicable)
    "currency",             # Currency code (e.g., USD)
    "size",                 # Size of the app in MB or GB (optional)
    "released",             # Release date of the app
    "last_updated",         # Last updated date of the app
    "content_rating",       # Content rating (e.g., Everyone, Teen)
    "privacy_policy",       # URL to the privacy policy
    "ad_supported",         # Whether the app is ad-supported or not
    "in_app_purchases",     # Whether the app supports in-app purchases or not
    "editors_choice"        # Whether the app is an editor's choice or not
]


<h4>Below Cell Inserts Dataset According to Designed Schema</h4>

In [ ]:
#df1 = df1.iloc[:100].copy() #for Test 

In [ ]:
from sqlalchemy import Column, Integer, String, Float, Boolean, DateTime, ForeignKey, BigInteger, Text
from sqlalchemy.orm import relationship,declarative_base

Base = declarative_base()

class Developer(Base):
    __tablename__ = 'developers'

    developer_id = Column(Integer, primary_key=True)
    developer_website = Column(String)
    developer_email = Column(String)

    # Relationship
    apps = relationship("App", back_populates="developer")

class Category(Base):
    __tablename__ = 'categories'

    category_id = Column(Integer, primary_key=True)
    category = Column(String, nullable=False)

    # Relationship
    apps = relationship("App", back_populates="category")

class App(Base):
    __tablename__ = 'apps'

    app_id = Column(Integer, primary_key=True)
    app_name = Column(String, nullable=False)
    developer_id = Column(Integer, ForeignKey('developers.developer_id'))
    category_id = Column(Integer, ForeignKey('categories.category_id'))
    rating = Column(Float)
    rating_count = Column(BigInteger)
    installs = Column(String)
    minimum_installs = Column(BigInteger)
    maximum_installs = Column(BigInteger)
    minimum_android = Column(String)
    free = Column(Boolean)
    price = Column(Float)
    currency = Column(String)
    size = Column(Float)
    released = Column(DateTime)
    last_updated = Column(DateTime)
    content_rating = Column(String)
    privacy_policy = Column(Text)
    ad_supported = Column(Boolean)
    in_app_purchases = Column(Boolean)
    editors_choice = Column(Boolean)

    # Relationships
    developer = relationship("Developer", back_populates="apps")
    category = relationship("Category", back_populates="apps")

Base.metadata.create_all(engine)


In [8]:
with engine.begin():

    #Insert Developers
    developers = df1.loc[:,developers_columns].drop_duplicates(subset=['developer_id'])
    developers.to_sql('developers', con = engine, index=False, if_exists='append', schema='googleplay', method='multi')

    #Insert categories
    categories = df1.loc[:,categories_columns].drop_duplicates(subset=['category_id'])
    categories.to_sql('categories', con = engine, index=False, if_exists='append', schema='googleplay', method='multi')

    #Insert Apps 
    apps = df1.loc[:,apps_columns]
    apps.to_sql('apps', con = engine, index=False, if_exists='append', schema='googleplay', method='multi',chunksize=10000)

